# Driving Behavior Predict and Visualize Importance

In [21]:
import os
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns

from contextlib import contextmanager
from time import time
from tqdm import tqdm
import lightgbm as lgbm
import category_encoders as ce

from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, log_loss, accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import matplotlib.dates as dates
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Data preparation

In [22]:
train0 = pd.read_csv("../input/driving-behavior/train_motion_data.csv")
test0 = pd.read_csv("../input/driving-behavior/test_motion_data.csv")
display(train0[0:3])
display(test0[0:3])

In [23]:
fig=make_subplots(specs=[[{"secondary_y":False}]])
fig.add_trace(go.Scatter(x=train0['Timestamp'],y=train0['AccX'],name='AccX'),secondary_y=False,)
fig.update_layout(autosize=False,width=700,height=400,title_text="Train AccX")
fig.update_xaxes(title_text="Timestamp")
fig.update_yaxes(title_text="Motion",secondary_y=False)
fig.show()

In [24]:
fig=make_subplots(specs=[[{"secondary_y":False}]])
fig.add_trace(go.Scatter(x=test0['Timestamp'],y=test0['AccX'],name='AccX'),secondary_y=False,)
fig.update_layout(autosize=False,width=700,height=400,title_text="Test AccX")
fig.update_xaxes(title_text="Timestamp")
fig.update_yaxes(title_text="Motion",secondary_y=False)
fig.show()

In [25]:
train0=train0.sample(frac=1, random_state=2021)
test0=test0.sample(frac=1, random_state=2021)
display(train0[0:3])
display(test0[0:3])

In [26]:
Name=sorted(train0['Class'].unique().tolist())
print(Name)
print(len(Name))
N=list(range(len(Name)))
normal_mapping=dict(zip(Name,N)) 
reverse_mapping=dict(zip(N,Name))        
train0['Class']=train0['Class'].map(normal_mapping)
print(normal_mapping)

In [27]:
print(train0.columns.tolist())

In [28]:
dummies =  pd.get_dummies(train0['Class'])
dummies.columns=Name
display(dummies)
train0=train0.drop(['Class'],axis=1)
train2=pd.concat([train0,dummies],axis=1)
display(train2)

In [29]:
tdummies =  pd.get_dummies(test0['Class'])
tdummies.columns=Name
display(tdummies)
test0=test0.drop(['Class'],axis=1)
test2=pd.concat([test0,tdummies],axis=1)
display(test2)

# Target setting

In [30]:
target=Name
trainY=train2[Name]
trainX=train2.drop(Name,axis=1)
testY=test2[Name]
testX=test2.drop(Name,axis=1)

In [31]:
df_columns = list(trainX.columns)
print(df_columns)

In [32]:
def create_numeric_feature(input_df):
    use_columns = df_columns 
    return input_df[use_columns].copy()

In [33]:
from contextlib import contextmanager
from time import time

class Timer:
    def __init__(self, logger=None, format_str='{:.3f}[s]', prefix=None, suffix=None, sep=' '):

        if prefix: format_str = str(prefix) + sep + format_str
        if suffix: format_str = format_str + sep + str(suffix)
        self.format_str = format_str
        self.logger = logger
        self.start = None
        self.end = None

    @property
    def duration(self):
        if self.end is None:
            return 0
        return self.end - self.start

    def __enter__(self):
        self.start = time()

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.end = time()
        out_str = self.format_str.format(self.duration)
        if self.logger:
            self.logger.info(out_str)
        else:
            print(out_str)

In [34]:
from tqdm import tqdm

def to_feature(input_df):

    processors = [
        create_numeric_feature,
    ]
    
    out_df = pd.DataFrame()
    
    for func in tqdm(processors, total=len(processors)):
        with Timer(prefix='create' + func.__name__ + ' '):
            _df = func(input_df)

        assert len(_df) == len(input_df), func.__name__
        out_df = pd.concat([out_df, _df], axis=1)
        
    return out_df

In [35]:
train_feat_df = to_feature(trainX)
test_feat_df = to_feature(testX)

# Model

In [36]:
import lightgbm as lgbm
from sklearn.metrics import mean_squared_error

def fit_lgbm(X, y, cv, 
             params: dict=None, 
             verbose: int=50):

    if params is None:
        params = {}

    models = []
    oof_pred = np.zeros_like(y, dtype=np.float)

    for i, (idx_train, idx_valid) in enumerate(cv): 
        x_train, y_train = X[idx_train], y[idx_train]
        x_valid, y_valid = X[idx_valid], y[idx_valid]

        clf = lgbm.LGBMRegressor(**params)
        
        with Timer(prefix='fit fold={} '.format(i)):
            clf.fit(x_train, y_train, 
                    eval_set=[(x_valid, y_valid)],  
                    early_stopping_rounds=100,
                    verbose=verbose)

        pred_i = clf.predict(x_valid)
        oof_pred[idx_valid] = pred_i
        models.append(clf)
        print(f'Fold {i} RMSLE: {mean_squared_error(y_valid, pred_i) ** .5:.4f}')
        print()

    score = mean_squared_error(y, oof_pred) ** .5
    print('-' * 50)
    print('FINISHED | Whole RMSLE: {:.4f}'.format(score))
    return oof_pred, models

In [37]:
params = {
    'objective': 'rmse', 
    'learning_rate': .1,
    'reg_lambda': 1.,
    'reg_alpha': .1,
    'max_depth': 5, 
    'n_estimators': 10000, 
    'colsample_bytree': .5, 
    'min_child_samples': 10,
    'subsample_freq': 3,
    'subsample': .9,
    'importance_type': 'gain', 
    'random_state': 71,
    'num_leaves': 62
}

In [38]:
y = trainY
print(y.shape)
print(type(y))

In [39]:
ydf=pd.DataFrame(y)
ydf

In [40]:
from sklearn.model_selection import KFold

for i in range(len(Name)):
    fold = KFold(n_splits=5, shuffle=True, random_state=71)
    ydfi=ydf.iloc[:,i]
    y=np.array(ydfi)
    cv = list(fold.split(train_feat_df, y))
    oof, models = fit_lgbm(train_feat_df.values, y, cv, params=params, verbose=500)
    
    fig,ax = plt.subplots(figsize=(6,6))
    ax.set_title(target[i],fontsize=20)
    ax.set_xlabel('Train Actual',fontsize=12)
    ax.set_ylabel('Train Predicted',fontsize=12)
    ax.scatter(y,oof)


# Visualize Importance

In [41]:
def visualize_importance(models, feat_train_df):

    feature_importance_df = pd.DataFrame()
    for i, model in enumerate(models):
        _df = pd.DataFrame()
        _df['feature_importance'] = model.feature_importances_
        _df['column'] = feat_train_df.columns
        _df['fold'] = i + 1
        feature_importance_df = pd.concat([feature_importance_df, _df], 
                                          axis=0, ignore_index=True)

    order = feature_importance_df.groupby('column')\
        .sum()[['feature_importance']]\
        .sort_values('feature_importance', ascending=False).index[:50]

    fig, ax = plt.subplots(figsize=(8, max(6, len(order) * .25)))
    sns.boxenplot(data=feature_importance_df, 
                  x='feature_importance', 
                  y='column', 
                  order=order, 
                  ax=ax, 
                  palette='viridis', 
                  orient='h')
    
    ax.tick_params(axis='x', rotation=0)
    #ax.set_title('Importance')
    ax.grid()
    fig.tight_layout()
    
    return fig,ax

#fig, ax = visualize_importance(models, train_feat_df)

In [42]:
PREDS=[]
for i in range(len(Name)):
    fold = KFold(n_splits=5, shuffle=True, random_state=71)
    ydfi=ydf.iloc[:,i]
    y=np.array(ydfi)
    cv = list(fold.split(train_feat_df, y))
    oof, models = fit_lgbm(train_feat_df.values, y, cv, params=params, verbose=500)
    fig, ax = visualize_importance(models, train_feat_df)
    ax.set_title(target[i]+' Imortance',fontsize=20)
    
    pred0 = np.array([model.predict(test_feat_df.values) for model in models])
    PREDS += [pred0[4]]

In [43]:
TName=['YES','NO']
for i in range(len(Name)):
    PREDi=PREDS[i]
    PREDi2=[]
    for item in PREDi:
        PREDi2+=[int(item)]

    ANSi=testY.iloc[:,i]
    ANSi2=[]
    for item in ANSi:
        ANSi2+=[item]
    
    print(Name[i])
    print(classification_report(np.array(ANSi2), np.array(PREDi2), target_names=TName, digits=4))